# **Chatbot Application using Deep Learning and PyTorch**

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from nltk.stem import PorterStemmer

In [3]:
import numpy as np

In [4]:
stemmer = PorterStemmer()

In [5]:
def tokenizer(sentence):
  return nltk.word_tokenize(sentence)

In [6]:
def stem(tokenized_sentence):
  return stemmer.stem(tokenized_sentence)

In [7]:
def bag_of_words(tokenized_sent, vocab):
  tokenized_sent = [stem(w) for w in tokenized_sent]
  bag = np.zeros(len(vocab), dtype = np.float32)
  for idx,w in enumerate(vocab):
    if w in tokenized_sent:
      bag[idx] = 1.0
  return bag

In [8]:
stemmed_words  = [stem(w) for w in tokenizer("Hello, how are you doing?")]
print(stemmed_words)

['hello', ',', 'how', 'are', 'you', 'do', '?']


In [9]:
#importing json data for the chatbot

import json

In [10]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/ChatBot")
os.listdir()

Mounted at /content/drive


['intents.json', 'NLTK.ipynb']

In [11]:
with open('intents.json') as json_file:
    intents = json.load(json_file)

In [12]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context_set': '',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'tag': 'goodbye'},
  {'context_set': '',
   'patterns': ['how old',
    'how old is tim',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': ['I am 27 years old!', '27 years young!'],
   'tag': 'age'},
  {'context_set': '',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Pointer.', "I'm pointer!"],
   'tag': 'name'},
  {'context_set': '',
   'patterns': ['Id like to buy something',
    'whats on the menu',
    'what do you reccommen

CREATE TRAINING DATA
Tokenize -> lowering+stem -> exclude punctuation chars -> Bag of Words

In [13]:
vocabulary = []
tags = []
data = []

In [14]:
for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for i in intent['patterns']:
    w = tokenizer(i)
    vocabulary.extend(w)
    data.append((w,tag))

In [15]:
tags

['greeting', 'goodbye', 'age', 'name', 'shop', 'hours']

In [16]:
punctuation = ['?',',','.','!']
vocabulary = [stem(w.lower()) for w in vocabulary if w not in punctuation]
vocabulary

['hi',
 'how',
 'are',
 'you',
 'is',
 'anyon',
 'there',
 'hello',
 'good',
 'day',
 'what',
 'up',
 'cya',
 'see',
 'you',
 'later',
 'goodby',
 'i',
 'am',
 'leav',
 'have',
 'a',
 'good',
 'day',
 'how',
 'old',
 'how',
 'old',
 'is',
 'tim',
 'what',
 'is',
 'your',
 'age',
 'how',
 'old',
 'are',
 'you',
 'age',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'i',
 'call',
 'you',
 'what',
 'your',
 'name',
 'id',
 'like',
 'to',
 'buy',
 'someth',
 'what',
 'on',
 'the',
 'menu',
 'what',
 'do',
 'you',
 'reccommend',
 'could',
 'i',
 'get',
 'someth',
 'to',
 'eat',
 'when',
 'are',
 'you',
 'guy',
 'open',
 'what',
 'are',
 'your',
 'hour',
 'hour',
 'of',
 'oper']

In [17]:
vocabulary = sorted(set(vocabulary))

In [18]:
tags = sorted(set(tags))

In [19]:
X_train=[]
y_train = []

In [20]:
for (pattern_sent,tag) in data:
  bag = bag_of_words(pattern_sent,vocabulary)
  X_train.append(bag)
  label = tags.index(tag)
  y_train.append(label)   #cross entropy loss doesnt need one-hot encoded labels

In [21]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [22]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Creating PyTorch dataset

In [23]:
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

In [24]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(X_train)
    self.x_data = X_train
    self.y_data = y_train
  
  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]
  
  def __len__(self):
    return self.n_samples
  
  

In [26]:
#hyperparameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate =0.001
num_epochs =1000

In [27]:
print(input_size, len(vocabulary))

47 47


In [28]:
print(output_size, tags)

6 ['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']


In [37]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [44]:
#Model training
#feed forward nn with 2 hidden layers
#bagof words as input to ip layer (number of different patterns as input size), 
#output layer equals number of different classes, then softmax activation
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size,hidden_size)  #creating 3 linear layers
    self.l2 = nn.Linear(hidden_size,hidden_size)
    self.l3 = nn.Linear(hidden_size,num_classes)

    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)

    out = self.l2(out)
    out = self.relu(out)

    out = self.l3(out)
    #no activation and no softmax

    return out





In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  #GPU availability check
print(device)

cuda


In [45]:
model = NeuralNet(input_size, hidden_size,output_size).to(device)

#loss and optimizer 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() ,lr = learning_rate)

In [46]:


for epoch in range(num_epochs):
  for (words,labels) in train_loader:
    words = words.to(device)
    labels = labels.to(device)

    #forward pass
    outputs = model(words)
    loss = criterion(outputs,labels)

    #backward and optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  if (epoch + 1)%100 == 0:
    print(f'epoch {epoch+1}/{num_epochs},loss = {loss.item():.4f}') 

print(f'final loss,loss = {loss.item():.4f}') 



epoch 100/1000,loss = 1.3864
epoch 200/1000,loss = 0.3182
epoch 300/1000,loss = 0.0447
epoch 400/1000,loss = 0.0053
epoch 500/1000,loss = 0.0055
epoch 600/1000,loss = 0.0020
epoch 700/1000,loss = 0.0045
epoch 800/1000,loss = 0.0019
epoch 900/1000,loss = 0.0009
epoch 1000/1000,loss = 0.0012
final loss,loss = 0.0012


In [47]:
data  = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hidden_size":hidden_size,
    "vocabulary":vocabulary,
    "tags":tags
}
 
FILE = "data.pth"
torch.save(data, FILE)

print(f'training completed, File saved to {FILE}')

training completed, File saved to data.pth


CHAT BOT 

In [48]:
import random 


In [49]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
vocab = data["vocabulary"]
tags = data["tags"]
model_state = data["model_state"]

In [52]:
model =  NeuralNet(input_size,hidden_size,output_size)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=47, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=6, bias=True)
  (relu): ReLU()
)

CREATING THE CHAT

In [53]:
bot_name = "Aravind"
print("Let's Talk! Type 'quit' to exit :)")

while True:
  sentence = input("You: ")
  if sentence == 'quit':
    break
  sentence = tokenizer(sentence)
  X = bag_of_words(sentence,vocab)
  X = X.reshape(1,X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _,predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output,dim=1)
  prob = probs[0][predicted.item()]

  if prob.item() > 0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(f'{bot_name}: {random.choice(intent["responses"])}')
  else:
    print(f'{bot_name}: Sorry, I do not understand :(')

Let's Talk! Type 'quit' to exit :)
You: hi
Aravind: Hi there, how can I help?
You: how are you
Aravind: Good to see you again!
You: opening time?
Aravind: Hello!
You: opening time?
Aravind: Good to see you again!
You: time?
Aravind: Good to see you again!
You: hour?
Aravind: Sorry, I do not understand :(
You: menu?
Aravind: Hi there, how can I help?
You: whats on the menu
Aravind: We sell chocolate chip cookies for $2!
You: opening?
Aravind: Hello!
You: open hour
Aravind: We are open 7am-4pm Monday-Friday!
You: quit
